In [ ]:
###### from datetime import date

from openfisca_france import init_country
from openfisca_france.model.base import *

In [ ]:
import functools

from openfisca_core.formulas import make_formula_decorator, neutralize_column
from openfisca_france.entities import entity_class_by_symbol

reference_formula = make_formula_decorator(entity_class_by_symbol = entity_class_by_symbol, update = True)
reference_input_variable = functools.partial(reference_input_variable, update = True)

# Système socio-fiscal

In [ ]:
TaxBenefitSystem = init_country()
tax_benefit_system = TaxBenefitSystem()

In [ ]:
from openfisca_core import reforms

In [ ]:
ReformeRevenuDeBase = reforms.make_reform(
    key = 'revenu_de_base_enfants',
    name = u"Réforme Revenu de base enfants",
    reference = tax_benefit_system,
    )

In [ ]:
@ReformeRevenuDeBase.formula
class qf_to_ind(EntityToPersonColumn):
    entity_class = Individus
    label = u"avantage quotient familial enfant : on le passe à l'individu"
    roles = [0]
    variable = FoyersFiscaux.column_by_name['avantage_qf']
    
@ReformeRevenuDeBase.formula
class qf_to_fam(PersonToEntityColumn):
    entity_class = Familles
    label = u"avantage quotient familial enfant : on le passe à la famille"
    roles = [0]
    variable = qf_to_ind
    
@ReformeRevenuDeBase.formula
class ecpess_to_ind(EntityToPersonColumn):
    entity_class = Individus
    label = u"ecpess : on le passe à l'individu"
    roles = [0]
    variable = FoyersFiscaux.column_by_name['ecpess']
    
@ReformeRevenuDeBase.formula
class ecpess_to_fam(PersonToEntityColumn):
    entity_class = Familles
    label = u"ecpess : on le passe à la famille"
    roles = [0]
    variable = ecpess_to_ind

In [ ]:
#ReformeRevenuDeBase.formula(neutralize_column(tax_benefit_system.column_by_name['af_base']))    # Suppression des allocations familiales        
#ReformeRevenuDeBase.formula(neutralize_column(tax_benefit_system.column_by_name['af']))    # Suppression des allocations familiales
#ReformeRevenuDeBase.formula(neutralize_column(tax_benefit_system.column_by_name['cf']))
#ReformeRevenuDeBase.formula(neutralize_column(tax_benefit_system.column_by_name['ars']))


@ReformeRevenuDeBase.formula
class aide_famille(SimpleFormulaColumn):
    column = FloatCol
    entity_class = Familles
    label = u"af_base + af + cf + ars + paje_base (manque ecpess)"

    def function(self, simulation, period):
        year = period.start.offset('first-of', 'year').period('year')
        af_base = simulation.calculate_add('af_base', year)
        af = simulation.calculate_add('af', year)
        cf = simulation.calculate_add('cf', year)
        ars = simulation.calculate('ars', year)
        paje_base = simulation.calculate_add('paje_base', year)
        qf = simulation.calculate('qf_to_fam', year)
        ecpess = simulation.calculate('ecpess_to_fam', year)
        
        return year, af_base + af + cf + paje_base + qf + ars + ecpess

# Réforme : 1. Revenu de base enfant

In [ ]:
'''
La redistribution monétaire en faveur des familles combine 8 mécanismes principaux, sous conditions de ressources :
- Allocations familiales – AF (de 0 à 19 ou 20 ans)
- Majoration pour âge des AF (à partir de 14 ans)
- Complément familial – CF (pour un 3ème enfant)
- Allocation de base de la PAJE (de 0 à 2 ans)
- Allocation de rentrée scolaire – ARS (de 6 à 18 ans)
- Supplément de RSA calculé par enfant (déduit AF, CF, PAJE)
- Diminution de l’impôt par le quotient familial – QF (plafonné)
- Réduction d’impôt pour scolarisation des enfants 

La réforme simulée ici permet de remplacer ces 8 dispositifs par la combinaison de :
- un Revenu de Base Enfant, pour chaque enfant de 0 à l'anniversaire de ses 14 ans
- un Revenu de Base Jeune, de 14 jusqu'à l'anniversaire de ses 25 ans

Divers montants de ces RdB doivent pouvoir être testés.
Un repère pour 2015 est 185€ pour le RdB Enfant / 230€ pour le RdB Jeune
'''

In [ ]:
from numpy import logical_not as not_, minimum as min_, maximum as max_

@ReformeRevenuDeBase.formula
class nbptr(SimpleFormulaColumn):
    reference = FoyersFiscaux.column_by_name['nbptr']

    # On enlève les enfants du calcul du nbptr (quotient_familial.enf*)
    
    def function(self, simulation, period):
        '''
        Nombre de parts du foyer
        'foy'
        note 1 enfants et résidence alternée (formulaire 2041 GV page 10)

        quotient_familial.conj : nb part associées au conjoint d'un couple marié ou pacsé
        quotient_familial.enf1 : nb part 2 premiers enfants
        quotient_familial.enf2 : nb part enfants de rang 3 ou plus
        quotient_familial.inv1 : nb part supp enfants invalides (I, G)
        quotient_familial.inv2 : nb part supp adultes invalides (R)
        quotient_familial.not31 : nb part supp note 3 : cases W ou G pour veuf, celib ou div
        quotient_familial.not32 : nb part supp note 3 : personne seule ayant élevé des enfants
        quotient_familial.not41 : nb part supp adultes invalides (vous et/ou conjoint) note 4
        quotient_familial.not42 : nb part supp adultes anciens combattants (vous et/ou conjoint) note 4
        quotient_familial.not6 : nb part supp note 6
        quotient_familial.isol : demi-part parent isolé (T)
        quotient_familial.edcd : enfant issu du mariage avec conjoint décédé;
        '''
        period = period.start.offset('first-of', 'month').period('year')
        nb_pac = simulation.calculate('nb_pac', period)
        marpac = simulation.calculate('marpac', period)
        celdiv = simulation.calculate('celdiv', period)
        veuf = simulation.calculate('veuf', period)
        jveuf = simulation.calculate('jveuf', period)
        nbF = simulation.calculate('nbF', period)
        nbG = simulation.calculate('nbG', period)
        nbH = simulation.calculate('nbH', period)
        nbI = simulation.calculate('nbI', period)
        nbR = simulation.calculate('nbR', period)
        nbJ = simulation.calculate('nbJ', period)
        caseP = simulation.calculate('caseP', period)
        caseW = simulation.calculate('caseW', period)
        caseG = simulation.calculate('caseG', period)
        caseE = simulation.calculate('caseE', period)
        caseK = simulation.calculate('caseK', period)
        caseN = simulation.calculate('caseN', period)
        caseF = simulation.calculate('caseF', period)
        caseS = simulation.calculate('caseS', period)
        caseL = simulation.calculate('caseL', period)
        caseT = simulation.calculate('caseT', period)
        quotient_familial = simulation.legislation_at(period.start).ir.quotient_familial

        no_pac = nb_pac == 0  # Aucune personne à charge en garde exclusive
        has_pac = not_(no_pac)
        no_alt = nbH == 0  # Aucun enfant à charge en garde alternée
        has_alt = not_(no_alt)

        # # nombre de parts liées aux enfants à charge
        # que des enfants en résidence alternée
        enf1 = (no_pac & has_alt) * (quotient_familial.enf1 * min_(nbH, 2) * 0.5
                                     + quotient_familial.enf2 * max_(nbH - 2, 0) * 0.5)
        # pas que des enfants en résidence alternée
        enf2 = (has_pac & has_alt) * ((nb_pac == 1) * (quotient_familial.enf1 * min_(nbH, 1) * 0.5
            + quotient_familial.enf2 * max_(nbH - 1, 0) * 0.5) + (nb_pac > 1) * (quotient_familial.enf2 * nbH * 0.5))
        # pas d'enfant en résidence alternée
        enf3 = quotient_familial.enf1 * min_(nb_pac, 2) + quotient_familial.enf2 * max_((nb_pac - 2), 0)

        enf = enf1 + enf2 + enf3
        # # note 2 : nombre de parts liées aux invalides (enfant + adulte)
        n2 = quotient_familial.inv1 * (nbG + nbI / 2) + quotient_familial.inv2 * nbR

        # # note 3 : Pas de personne à charge
        # - invalide

        n31a = quotient_familial.not31a * (no_pac & no_alt & caseP)
        # - ancien combatant
        n31b = quotient_familial.not31b * (no_pac & no_alt & (caseW | caseG))
        n31 = max_(n31a, n31b)
        # - personne seule ayant élevé des enfants
        n32 = quotient_familial.not32 * (no_pac & no_alt & ((caseE | caseK) & not_(caseN)))
        n3 = max_(n31, n32)
        # # note 4 Invalidité de la personne ou du conjoint pour les mariés ou
        # # jeunes veuf(ve)s
        n4 = max_(quotient_familial.not41 * (1 * caseP + 1 * caseF), quotient_familial.not42 * (caseW | caseS))

        # # note 5
        #  - enfant du conjoint décédé
        n51 = quotient_familial.cdcd * (caseL & ((nbF + nbJ) > 0))
        #  - enfant autre et parent isolé
        n52 = quotient_familial.isol * caseT * (((no_pac & has_alt) * ((nbH == 1) * 0.5 + (nbH >= 2))) + 1 * has_pac)
        n5 = max_(n51, n52)

        # # note 6 invalide avec personne à charge
        n6 = quotient_familial.not6 * (caseP & (has_pac | has_alt))

        # # note 7 Parent isolé
        n7 = quotient_familial.isol * caseT * ((no_pac & has_alt) * ((nbH == 1) * 0.5 + (nbH >= 2)) + 1 * has_pac)

        # # Régime des mariés ou pacsés
        #m = 1 + quotient_familial.conj + enf + n2 + n4
        m = 1 + quotient_familial.conj + n2 + n4
        
        # # veufs  hors jveuf
        # v = 1 + enf + n2 + n3 + n5 + n6
        v = 1 + n2 + n3 + n5 + n6
        
        # # celib div
        #c = 1 + enf + n2 + n3 + n6 + n7
        c = 1 + n2 + n3 + n6 + n7

        return period, (marpac | jveuf) * m + (veuf & not_(jveuf)) * v + celdiv * c


In [ ]:
# Suppression du nombre d'enfants dans le calcul du RSA socle

@ReformeRevenuDeBase.formula
class rsa_socle(SimpleFormulaColumn):
    reference = Familles.column_by_name['rsa_socle']

    def function(self, simulation, period):
        period = period.start.offset('first-of', 'month').period('month')
        age_holder = simulation.compute('age', period)
        smic55_holder = simulation.compute('smic55', period)
        activite_holder = simulation.compute('activite', period)
        nb_par = simulation.calculate('nb_par', period)
        rmi = simulation.legislation_at(period.start).minim.rmi

        age_parents = self.split_by_roles(age_holder, roles = [CHEF, PART])
        activite_parents = self.split_by_roles(activite_holder, roles = [CHEF, PART])
        age_enf = self.split_by_roles(age_holder, roles = ENFS)
        smic55_enf = self.split_by_roles(smic55_holder, roles = ENFS)

        nbp = nb_par

        eligib = (
            (age_parents[CHEF] >= rmi.age_pac)
            *
            not_(activite_parents[CHEF] == 2)
            ) | (
                (age_parents[PART] >= rmi.age_pac) * not_(activite_parents[PART] == 2)
                )

        taux = (
            1 + (nbp >= 2) * rmi.txp2 +
            (nbp >= 3) * rmi.txp3 +
            (nbp >= 4) * ((nb_par == 1) * rmi.txps + (nb_par != 1) * rmi.txp3) +
            max_(nbp - 4, 0) * rmi.txps
            )
        return period, eligib * rmi.rmi * taux

In [ ]:
# Suppression du nombre d'enfants dans le calcul du RSA forfait logement

@ReformeRevenuDeBase.formula
class rmi_nbp(SimpleFormulaColumn):
    reference = Familles.column_by_name['rmi_nbp']

    def function(self, simulation, period):
        period = period.start.offset('first-of', 'month').period('month')
        age_holder = simulation.compute('age', period)
        smic55_holder = simulation.compute('smic55', period)
        nb_par = simulation.calculate('nb_par', period)
        P = simulation.legislation_at(period.start).minim.rmi

        age = self.split_by_roles(age_holder, roles = ENFS)
        smic55 = self.split_by_roles(smic55_holder, roles = ENFS)

        return period, nb_par # + nb_enf(age, smic55, 0, P.age_pac - 1)


In [ ]:
# Création d'un revenu de base enfant - Version famille

from openfisca_france.model.prestations.prestations_familiales.base_ressource import nb_enf

@ReformeRevenuDeBase.formula
class rdb_enfant_famille(SimpleFormulaColumn):
    column = FloatCol
    entity_class = Familles
    label = u"Revenu de base enfant"

    def function(self, simulation, period):
        period = period.start.offset('first-of', 'month').period('month')
        age_holder = simulation.compute('age', period)
        P = simulation.legislation_at(period.start).fam.af        
        bmaf = P.bmaf

        
        smic55_holder = simulation.compute('smic55', period)
        smic55 = self.split_by_roles(smic55_holder, roles = ENFS)        
        age = self.split_by_roles(age_holder, roles = ENFS)
        
        smic5 = {
            role: array * 0
            for role, array in smic55.iteritems()
            }
        nbenf_inf13 = nb_enf(age, smic5, 0, 13)
        nbenf_sup14 = nb_enf(age, smic5, 14, 18)
        
        return period, (nbenf_inf13 * 0.41 + nbenf_sup14 * 0.57) * bmaf

# Les taux 0,41 et 0,16 (0,57-0,41) sont issus des allocations familiales


In [ ]:
# Création d'un revenu de base enfant - Version individus

@ReformeRevenuDeBase.formula
class rdb_enf(SimpleFormulaColumn):
    column = FloatCol
    entity_class = Individus
    label = u"Revenu de base enfant"

    def function(self, simulation, period):
        period = period.start.offset('first-of', 'month').period('month')
        age = simulation.calculate('age')
        P = simulation.legislation_at(period.start).fam.af        
        bmaf = P.bmaf
        
        return period, ((age < 14) * 0.41 + not_(age < 14) * 0.57) * bmaf * (age <= 18)

In [ ]:
@ReformeRevenuDeBase.formula
class revdisp(SimpleFormulaColumn):
    reference = Menages.column_by_name['revdisp']

    def function(self, simulation, period):
        '''
        Revenu disponible - ménage
        'men'
        '''
        period = period.start.offset('first-of', 'month').period('year')
        rev_trav_holder = simulation.compute('rev_trav', period)
        pen_holder = simulation.compute('pen', period)
        rev_cap_holder = simulation.compute('rev_cap', period)
        psoc_holder = simulation.compute('psoc', period)
        ppe_holder = simulation.compute('ppe', period)
        impo = simulation.calculate('impo', period)
        rdb_enfant_holder = simulation.compute_add('rdb_enf', period)

        pen = self.sum_by_entity(pen_holder)
        ppe = self.cast_from_entity_to_role(ppe_holder, role = VOUS)
        ppe = self.sum_by_entity(ppe)
        psoc = self.cast_from_entity_to_role(psoc_holder, role = CHEF)
        psoc = self.sum_by_entity(psoc)
        rev_cap = self.sum_by_entity(rev_cap_holder)
        rev_trav = self.sum_by_entity(rev_trav_holder)
        rdb_enfant = self.sum_by_entity(rdb_enfant_holder)

        return period, rev_trav + pen + rev_cap + psoc + ppe + impo + rdb_enfant

In [ ]:
reform = ReformeRevenuDeBase()

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import date  # module nécessaire pour la définition des dates, dont notamment les dates de naissances
from openfisca_france.tests.base import tax_benefit_system




def make_couple_with_child_scenario(nombre_enfants = 0, year = None, tax_benefit_system = tax_benefit_system,
                                    axes_variable = 'salaire_de_base',  ax_variable_max = 150000, count = 5000):
    enfant = [dict(
        birth = date(2005, 1, 1),
        )]
    enfants = enfant * nombre_enfants
    scenario = tax_benefit_system.new_scenario().init_single_entity(
        axes = [[
            dict(
                count = count,
                min = 0,
                max = ax_variable_max,
                name = axes_variable,
                period = year-2,
                ),
            dict(
                count = count,
                min = 0,
                max = ax_variable_max,
                name = axes_variable,
                period = year-1,
                ),
            dict(
                count = count,
                min = 0,
                max = ax_variable_max,
                name = axes_variable,
                period = year,
                ),
            ]],
        period = year,
        parent1 = dict(
            birth = date(1980, 1, 1),
            statmarit = 5, #pacsés
            ),
        parent2 = dict(
            birth = date(1980, 1, 1),
            statmarit = 5,
            ),
        enfants = enfants,
        menage = dict(
            loyer = 1000,
            statut_occupation = 4,
            ),
        )
    return scenario
def make_single_with_child_scenario(nombre_enfants = 1, year = None, tax_benefit_system = tax_benefit_system,
                                    axes_variable = 'salaire_de_base',  ax_variable_max = 150000, count = 5000):
    enfant = [dict(
        birth = date(2005, 1, 1),
        )]
    enfants = enfant * nombre_enfants
    scenario = tax_benefit_system.new_scenario().init_single_entity(
        axes = [[
            dict(
                count = count,
                min = 0,
                max = ax_variable_max,
                name = axes_variable,
                period = year-2,
                ),
            dict(
                count = count,
                min = 0,
                max = ax_variable_max,
                name = axes_variable,
                period = year-1,
                ),
            dict(
                count = count,
                min = 0,
                max = ax_variable_max,
                name = axes_variable,
                period = year,
                ),
            ]],
        period = year,
        parent1 = dict(
            birth = date(1980, 1, 1),
            ),
        enfants = enfants,
        menage = dict(
            loyer = 1000,
            statut_occupation = 4,
            ),
        )
    return scenario

In [ ]:
scenario_couple_avec_1_enfant = make_couple_with_child_scenario(1, 2014)
simulation_couple_avec_1_enfant = scenario_couple_avec_1_enfant.new_simulation()

scenario_couple_avec_2_enfants = make_couple_with_child_scenario(2, 2014)
simulation_couple_avec_2_enfants = scenario_couple_avec_2_enfants.new_simulation()

scenario_couple_avec_3_enfants = make_couple_with_child_scenario(3, 2014)
simulation_couple_avec_3_enfants = scenario_couple_avec_3_enfants.new_simulation()

scenario_couple_avec_4_enfants = make_couple_with_child_scenario(4, 2014)
simulation_couple_avec_4_enfants = scenario_couple_avec_4_enfants.new_simulation()

scenario_couple_avec_5_enfants = make_couple_with_child_scenario(5, 2014)
simulation_couple_avec_5_enfants = scenario_couple_avec_5_enfants.new_simulation()

In [ ]:
salaire_de_base = simulation_couple_avec_1_enfant.calculate_add("salaire_de_base")
aide_famille_1 = simulation_couple_avec_1_enfant.calculate_add("aide_famille")
aide_famille_2 = simulation_couple_avec_2_enfants.calculate_add("aide_famille")
aide_famille_3 = simulation_couple_avec_3_enfants.calculate_add("aide_famille")
aide_famille_4 = simulation_couple_avec_4_enfants.calculate_add("aide_famille")
aide_famille_5 = simulation_couple_avec_5_enfants.calculate_add("aide_famille")

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(salaire_de_base[::3], aide_famille_1)
plt.plot(salaire_de_base[::3], aide_famille_2)
plt.plot(salaire_de_base[::3], aide_famille_3)
plt.plot(salaire_de_base[::3], aide_famille_4)
plt.plot(salaire_de_base[::3], aide_famille_5)

In [ ]:
qf_to_fam_1 = simulation_couple_avec_1_enfant.calculate_add("avantage_qf")
qf_to_fam_2 = simulation_couple_avec_2_enfants.calculate_add("avantage_qf")
qf_to_fam_3 = simulation_couple_avec_3_enfants.calculate_add("avantage_qf")
qf_to_fam_4 = simulation_couple_avec_4_enfants.calculate_add("avantage_qf")
qf_to_fam_5 = simulation_couple_avec_5_enfants.calculate_add("avantage_qf")

In [ ]:
plt.plot(salaire_de_base[::3], qf_to_fam_1)
plt.plot(salaire_de_base[::3], qf_to_fam_2)
plt.plot(salaire_de_base[::3], qf_to_fam_3)
plt.plot(salaire_de_base[::3], qf_to_fam_4)
plt.plot(salaire_de_base[::3], qf_to_fam_5)

In [ ]:
scenario_couple_sans_enfant = make_couple_with_child_scenario(0, 2014, reform)
simulation_couple_sans_enfant = scenario_couple_sans_enfant.new_simulation()

In [ ]:
revdisp_0 = simulation_couple_sans_enfant.calculate_add("revdisp")
revdisp_1 = simulation_couple_avec_1_enfant.calculate_add("revdisp")
revdisp_2 = simulation_couple_avec_2_enfants.calculate_add("revdisp")
revdisp_3 = simulation_couple_avec_3_enfants.calculate_add("revdisp")
revdisp_4 = simulation_couple_avec_4_enfants.calculate_add("revdisp")
revdisp_5 = simulation_couple_avec_5_enfants.calculate_add("revdisp")

In [ ]:
diff_revdisp_1 = revdisp_1 - revdisp_0
diff_revdisp_2 = revdisp_2 - revdisp_0
diff_revdisp_3 = revdisp_3 - revdisp_0
diff_revdisp_4 = revdisp_4 - revdisp_0
diff_revdisp_5 = revdisp_5 - revdisp_0

In [ ]:
plt.plot(salaire_de_base[::3], diff_revdisp_1)
plt.plot(salaire_de_base[::3], diff_revdisp_2)
plt.plot(salaire_de_base[::3], diff_revdisp_3)
plt.plot(salaire_de_base[::3], diff_revdisp_4)
plt.plot(salaire_de_base[::3], diff_revdisp_5)

In [ ]:
irpp_0 = simulation_couple_sans_enfant.calculate_add("irpp")
irpp_1 = simulation_couple_avec_1_enfant.calculate_add("irpp")
irpp_2 = simulation_couple_avec_2_enfants.calculate_add("irpp")
irpp_3 = simulation_couple_avec_3_enfants.calculate_add("irpp")
irpp_4 = simulation_couple_avec_4_enfants.calculate_add("irpp")
irpp_5 = simulation_couple_avec_5_enfants.calculate_add("irpp")

In [ ]:
plt.plot(salaire_de_base[::3], irpp_1)
plt.plot(salaire_de_base[::3], irpp_2)
plt.plot(salaire_de_base[::3], irpp_3)
plt.plot(salaire_de_base[::3], irpp_4)
plt.plot(salaire_de_base[::3], irpp_5)

# Tests

In [ ]:
reform = ReformeRevenuDeBase()

In [ ]:
simulation = tax_benefit_system.new_scenario().init_single_entity(
    period = 2014,
    parent1 = dict(
        birth = date(1980, 1, 1),
        salaire_imposable = 12000,
        statmarit = u'Marié',
        ),
    parent2 = dict(
        birth = date(1980, 1, 1),
        salaire_imposable = 46000,
        statmarit = u'Marié',
        ),
    enfants = [
        dict(
            birth = date(2010, 1, 1),
            ),
        dict(
            birth = date(2005, 1, 1),
            ),
        dict(
            birth = date(1999, 1, 1),
            ),
        ],
    ).new_simulation(debug = True)

In [ ]:
simulation_pauvre = tax_benefit_system.new_scenario().init_single_entity(
    period = 2014,
    parent1 = dict(
        birth = date(1980, 1, 1),
        salaire_imposable = 12000,
        statmarit = u'Marié',
        ),
    parent2 = dict(
        birth = date(1980, 1, 1),
        salaire_imposable = 6000,
        statmarit = u'Marié',
        ),
    enfants = [
        dict(
            birth = date(2010, 1, 1),
            ),
        dict(
            birth = date(2005, 1, 1),
            ),
        dict(
            birth = date(1999, 1, 1),
            ),
        ],
    ).new_simulation(debug = True)

In [ ]:
reform_simulation = reform.new_scenario().init_single_entity(
    period = 2014,
    parent1 = dict(
        birth = date(1980, 1, 1),
        salaire_imposable = 12000,
        statmarit = u'Marié',
        ),
    parent2 = dict(
        birth = date(1980, 1, 1),
        salaire_imposable = 46000,
        statmarit = u'Marié',
        ),
    enfants = [
        dict(
            birth = date(2010, 1, 1),
            ),
        dict(
            birth = date(2005, 1, 1),
            ),
        dict(
            birth = date(1999, 1, 1),
            ),
        ],
    ).new_simulation(debug = True)

In [ ]:
reform_simulation_pauvre = reform.new_scenario().init_single_entity(
    period = 2014,
    parent1 = dict(
        birth = date(1980, 1, 1),
        salaire_imposable = 12000,
        statmarit = u'Marié',
        ),
    parent2 = dict(
        birth = date(1980, 1, 1),
        salaire_imposable = 6000,
        statmarit = u'Marié',
        ),
    enfants = [
        dict(
            birth = date(2010, 1, 1),
            ),
        dict(
            birth = date(2005, 1, 1),
            ),
        dict(
            birth = date(1999, 1, 1),
            ),
        ],
    ).new_simulation(debug = True)

In [ ]:
simulation.calculate('nbptr')

In [ ]:
reform_simulation.calculate('nbptr')

In [ ]:
simulation.calculate_add('af')

In [ ]:
reform_simulation.calculate_add('af')

In [ ]:
simulation.calculate_add('cf')

In [ ]:
reform_simulation.calculate_add('cf')

In [ ]:
simulation.calculate('ars')

In [ ]:
reform_simulation.calculate('ars')

In [ ]:
simulation.calculate_add('rsa')

In [ ]:
reform_simulation.calculate_add('rsa')

In [ ]:
simulation.calculate('rmi_nbp', period = '2014-01')

In [ ]:
reform_simulation.calculate_add('rmi_nbp', period = '2014-01')

In [ ]:
reform_simulation.calculate_add('rdb_enfant_famille')

In [ ]:
reform_simulation.calculate_add('rdb_enf')

In [ ]:
simulation.calculate('revdisp')

In [ ]:
reform_simulation.calculate('revdisp')

In [ ]:
simulation_pauvre.calculate('revdisp')

In [ ]:
reform_simulation_pauvre.calculate('revdisp')

In [ ]:
simulation.calculate('salaire_imposable')

In [ ]:
reform_simulation.calculate('salaire_imposable')

In [ ]:
simulation.calculate('salsuperbrut')

In [ ]:
reform_simulation.calculate('salsuperbrut')

In [ ]:
simulation.calculate('salaire_net')

In [ ]:
reform_simulation.calculate('salaire_net')

In [ ]:
simulation.calculate('pfam')

In [ ]:
reform_simulation.calculate('pfam')

In [ ]:
simulation_pauvre.calculate('psoc')

In [ ]:
simulation.calculate('cotisations_salariales')

In [ ]:
reform_simulation_pauvre.calculate('pfam')

In [ ]:
simulation.calculate('irpp')

In [ ]:
reform_simulation.calculate('irpp')

In [ ]:
simulation.calculate('allegement_fillon')